In [0]:
%sql
-- Create structure
CREATE CATALOG ecommerce;
USE CATALOG ecommerce;
CREATE SCHEMA bronze;
CREATE SCHEMA silver;
CREATE SCHEMA gold;



In [0]:
%sql
CREATE VOLUME IF NOT EXISTS volumes_ecommerce;

In [0]:
from pyspark.sql.types import *

schema = StructType([
    StructField("event_time", TimestampType()),
    StructField("event_type", StringType()),
    StructField("product_id", StringType()),
    StructField("category_code", StringType()),
    StructField("brand", StringType()),
    StructField("price", DoubleType()),
    StructField("user_id", StringType()),
    StructField("user_session", StringType())
])

empty_df = spark.createDataFrame([], schema)


In [0]:
empty_df.write \
  .format("delta") \
  .mode("overwrite") \
  .save("/Volumes/ecommerce/default/volumes_ecommerce/bronze/events")


In [0]:
%sql
CREATE TABLE ecommerce.bronze.events
USING DELTA
AS SELECT * FROM delta.`/Volumes/ecommerce/default/volumes_ecommerce/bronze/events`
WHERE 1 = 0;


In [0]:
%sql
SELECT COUNT(*) FROM ecommerce.bronze.events;


In [0]:
empty_df.write \
  .format("delta") \
  .mode("overwrite") \
  .save("/Volumes/ecommerce/default/volumes_ecommerce/silver/events")


In [0]:
%sql
CREATE TABLE ecommerce.silver.events
USING DELTA
AS SELECT * FROM delta.`/Volumes/ecommerce/default/volumes_ecommerce/silver/events`
WHERE 1 = 0;


In [0]:
%sql
select * from ecommerce.silver.events

In [0]:
empty_df.write \
  .format("delta") \
  .mode("overwrite") \
  .save("/Volumes/ecommerce/default/volumes_ecommerce/gold/products")

In [0]:
%sql
CREATE TABLE ecommerce.gold.products
USING DELTA
AS SELECT * FROM delta.`/Volumes/ecommerce/default/volumes_ecommerce/gold/products`
WHERE 1 = 0;

In [0]:
%sql
GRANT SELECT ON TABLE gold.products TO `analysts@company.com`;
GRANT ALL PRIVILEGES ON SCHEMA silver TO `engineers@company.com`;

In [0]:
%sql

-- Controlled view
CREATE VIEW gold.top_products AS
SELECT brand, product_id
FROM gold.products
WHERE brand="apple" 
 LIMIT 100;